In [1]:
cd ..

E:\gdrive\Projects\Porto_Seguro\stacking


In [ ]:
import numpy as np
import pandas as pd
from rgf.sklearn import RGFClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import time
import subprocess
import glob

from utils import utils

In [ ]:
combined = utils.load_data()

In [9]:
features = ["ps_car_13", "ps_reg_03", "ps_ind_05_cat", "ps_ind_03", "ps_ind_15", "ps_reg_02", "ps_car_14", "ps_car_12", "ps_car_01_cat", 
"ps_car_07_cat","ps_ind_17_bin", "ps_car_03_cat", "ps_reg_01", "ps_car_15", "ps_ind_01", "ps_ind_16_bin", "ps_ind_07_bin", "ps_car_06_cat", 
"ps_car_04_cat", "ps_ind_06_bin", "ps_car_09_cat", "ps_car_02_cat", "ps_ind_02_cat", "ps_car_11", "ps_car_05_cat", "ps_calc_09", "ps_calc_05", 
"ps_ind_08_bin", "ps_car_08_cat", "ps_ind_09_bin", "ps_ind_04_cat", "ps_ind_18_bin", "ps_ind_12_bin", "ps_ind_14"]
combs = [('ps_reg_01', 'ps_car_02_cat'),  ('ps_reg_01', 'ps_car_04_cat')]

In [10]:
combined = combined[features]

In [4]:
combined.shape

(1488028, 57)

In [5]:
combined = utils.bojan_engineer(combined)

NameError: name 'name1' is not defined

In [12]:
for f1, f2 in combs:
        name = f1 + "_plus_" + f2
        df[name1] = df[f1].apply(lambda x: str(x)) + "_" + df[f2].apply(lambda x: str(x))
        lbl = LabelEncoder()
        lbl.fit(list(df[name].values))
        df[name1] = lbl.transform(list(df[name1].values))

In [14]:
a.nunique()

100

In [5]:
# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


def target_encode(trn_series=None,    # Revised to encode validation series
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)
    
# Read data
train_df = pd.read_csv('data/train.csv', na_values="-1") # .iloc[0:200,:]
test_df = pd.read_csv('data/test.csv', na_values="-1")

# from olivier
train_features = ["ps_car_13", "ps_reg_03", "ps_ind_05_cat", "ps_ind_03", "ps_ind_15", "ps_reg_02", "ps_car_14", "ps_car_12", "ps_car_01_cat", "ps_car_07_cat","ps_ind_17_bin", "ps_car_03_cat", "ps_reg_01", "ps_car_15", "ps_ind_01", "ps_ind_16_bin", "ps_ind_07_bin", "ps_car_06_cat", "ps_car_04_cat", "ps_ind_06_bin", "ps_car_09_cat", "ps_car_02_cat", "ps_ind_02_cat", "ps_car_11", "ps_car_05_cat", "ps_calc_09", "ps_calc_05",  "ps_ind_08_bin", "ps_car_08_cat", "ps_ind_09_bin", "ps_ind_04_cat", "ps_ind_18_bin", "ps_ind_12_bin", "ps_ind_14"]
# add combinations
combs = [('ps_reg_01', 'ps_car_02_cat'),  ('ps_reg_01', 'ps_car_04_cat')]

X = train_df[train_features]
test_df = test_df[train_features]

start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)
    


f_cats = [f for f in X.columns if "_cat" in f]

# Set up folds
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)
    

for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print( "\nFold ", i)
    
    # Enocode data
    for f in f_cats:
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_valid[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    # Run model for this fold
    X_train = X_train.fillna(X_train.mean())
    X_valid = X_valid.fillna(X_valid.mean())
    X_test = X_test.fillna(X_test.mean())

current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0
Fold  0

Fold  1

Fold  2

Fold  3

Fold  4


In [18]:
a = X_train.copy()

In [9]:
X_train.ps_car_09_cat_avg.nunique()

6

In [12]:
X_train.ps_reg_01_plus_ps_car_04_cat_avg.nunique()

95

In [13]:
X_train.ps_reg_01.nunique()

10

In [15]:
X_train.ps_car_04_cat.nunique()

10

In [27]:
# Read data
train = pd.read_csv('data/train.csv', na_values="-1") # .iloc[0:200,:]
test = pd.read_csv('data/test.csv', na_values="-1")
y = train.target

In [30]:
X_train['ps_car_04_cat' + "_avg"], X_val['ps_car_04_cat' + "_avg"], X_test['ps_car_04_cat' + "_avg"] = target_encode(
                                                trn_series=X_train['ps_car_04_cat'],
                                                val_series=X_val['ps_car_04_cat'],
                                                tst_series=X_test['ps_car_04_cat'],
                                                target=y_train,
                                                min_samples_leaf=200,
                                                smoothing=10,
                                                noise_level=0
                                                )

In [31]:
def bojan_engineer(df):
	# from olivier
	features = ["ps_car_13", "ps_reg_03", "ps_ind_05_cat", "ps_ind_03", "ps_ind_15", "ps_reg_02", "ps_car_14", "ps_car_12", "ps_car_01_cat", 
	"ps_car_07_cat","ps_ind_17_bin", "ps_car_03_cat", "ps_reg_01", "ps_car_15", "ps_ind_01", "ps_ind_16_bin", "ps_ind_07_bin", "ps_car_06_cat", 
	"ps_car_04_cat", "ps_ind_06_bin", "ps_car_09_cat", "ps_car_02_cat", "ps_ind_02_cat", "ps_car_11", "ps_car_05_cat", "ps_calc_09", "ps_calc_05", 
	"ps_ind_08_bin", "ps_car_08_cat", "ps_ind_09_bin", "ps_ind_04_cat", "ps_ind_18_bin", "ps_ind_12_bin", "ps_ind_14"]

	# add combinations
	combs = [('ps_reg_01', 'ps_car_02_cat'),  ('ps_reg_01', 'ps_car_04_cat')]

	df = df[features]

	for f1, f2 in combs:
	    name = f1 + "_plus_" + f2
	    
	    df[name] = df[f1].apply(lambda x: str(x)) + "_" + df[f2].apply(lambda x: str(x))
	    # Label Encode
	    df = pd.get_dummies(df, columns=[name])

	return df

def fold_smoothing_encode(X_train, X_val, X_test):
    for f in X_train.columns[X_train.columns.str.endswith('cat')==True]:
        X_train[f + "_avg"], X_val[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_val[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    return X_train, X_val, X_test


train = bojan_engineer(train)
test = bojan_engineer(test)

# Set up folds
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)
    

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    # Create data for this fold
    y_train, y_val = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_val = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()

    print( "\nFold ", i)
    
    X_train, X_val, X_test = fold_smoothing_encode(X_train, X_val, X_test)

    # Run model for this fold
    X_train = X_train.fillna(X_train.mean())
    X_val = X_val.fillna(X_val.mean())
    X_test = X_test.fillna(X_test.mean())


Fold  0

Fold  1

Fold  2

Fold  3

Fold  4


In [35]:
X_train.shape

(476170, 50)